## import libraries

In [6]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import sys
import pandas as pd
import scipy as sp

# sys.path.append('scripts/')
from scripts.ml_method import *
from scripts.proj1_helpers import *
from scripts.preprocess import *
from scripts.split_data import *
from scripts.model_testing import *


from sklearn.linear_model import LogisticRegression
from sklearn.decomposition import PCA


%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load data

In [159]:
y, x, ind = load_csv_data('higgs-data/train.csv')

In [160]:
# mask = x[:,4] == -999

# # mask = ((x[:,4] != -999) & (y == -1))
# # mask1 = ((x[:,4] == -999) & (y == -1))
# # mask2 = 
# np.random.choice(x[((x[:,4] != -999) & (y == -1)),4])

miss_list = [0, 4, 5, 6, 12, 23, 24, 25, 26, 27, 28]
resol = 10
dis_bin_b = np.zeros([len(miss_list), resol])
dis_bin_s = np.zeros([len(miss_list), resol])
min_max_b = np.zeros([len(miss_list),2])
min_max_s = np.zeros([len(miss_list),2])

                      
len_y = len(y)

for ind_i, i in enumerate(miss_list):
    # b's distribution 
    mask_b = ((x[:,i] == -999) & (y == -1))
    mask_s = ((x[:,i] == -999) & (y == 1))
    dis_b = x[((x[:,i] != -999) & (y == -1)),i]
    dis_s = x[((x[:,i] != -999) & (y == 1)),i]
    dis = x[(x[:,i] != -999),i]
    mask = (x[:,i] == -999) & (y == -1)
    
    x[mask, i] = np.random.choice(dis, len(x[mask,i]))
#     x[mask_b, i] = np.random.choice(dis_b, len(x[mask_b, i]))
#     x[mask_s, i] = np.random.choice(dis_s, len(x[mask_s, i]))

    min_max_b[ind_i, 1] = max_b = dis_b.max()
    min_max_b[ind_i, 0] = min_b = dis_b.min()
    min_max_s[ind_i, 1] = max_s = dis_s.max()
    min_max_s[ind_i, 0] = min_s = dis_s.min()
    width_b = (max_b - min_b)/resol
    width_s = (max_s - min_s)/resol
    
    x_tmp = x[:, i]
    
    for j in range(resol):
        dis_bin_b[ind_i][j] = len(x_tmp[(x_tmp >= (min_b + j * width_b)) & (x_tmp <= (min_b + (j+1) * width_b))])/len_y
        dis_bin_s[ind_i][j] = len(x_tmp[(x_tmp >= (min_s + j * width_s)) & (x_tmp <= (min_s + (j+1) * width_s))])/len_y

    
# x[:,5]
# x[:,4]

## Clean data and normalize data

In [161]:
stand_x = standardize(x, with_ones = True)

In [140]:
stand_x = getPCA(stand_x, 25)

## Split data

In [162]:
x_tr, x_te, y_tr, y_te = split_data(stand_x, y, 0.8, myseed=1)

## Training

In [163]:
losses, w = logistic_regression(y_tr, x_tr , np.zeros(stand_x.shape[1]),300, 0.000005)

y_pred = predict_labels(w, x_te)

np.mean(y_pred == y_te)

0.74741999999999997

## Cross validation

In [164]:
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn import svm

x_tr, x_te, y_tr, y_te = train_test_split(stand_x, y, test_size = 0.9, random_state = 0)

# x_tr.shape
# y_tr.shape

clf = svm.SVC(kernel='rbf', C=1).fit(x_tr, y_tr)
clf.score(x_te, y_te)
# --------- Split the data into 5 parts -----------

0.93333777777777782

In [ ]:
clf1 = clf

In [ ]:
def getIndex(value, ind):
    # for finding b's index
    ind_b = ind_s = 0
    
    if value < min_max_b[ind, 0]:
        ind_b = 0
    else:
        min(resol - 1, np.floor((value - min_max_b[ind, 0])/resol).astype(int))
    
    if value < min_max_s[ind, 0]:
        ind_s = 0
    else:
        min(resol - 1, np.floor((value - min_max_s[ind, 0])/resol).astype(int))
        
#     if dis_bin_b[ind, ind_b] > dis_bin_s[ind, ind_s]:
#         return 

In [126]:
min(3,2)

2

In [154]:
y1, x1, ind = load_csv_data('higgs-data/test.csv')

miss_list = [0, 4, 5, 6, 12, 23, 24, 25, 26, 27, 28]

for ind_i, i in enumerate(miss_list):
    # b's distribution 
    mask = ((x1[:,i] == -999))
    dis_b = x[((x[:,i] != -999) & (y == -1)),i]
    x1[mask, i] = np.random.choice(dis_b, len(x1[mask, i]))

stand_x = standardize(x1, with_ones = True)


In [155]:
k = clf.predict(stand_x)

In [156]:
create_csv_submission(ind, k, 'prediction.csv')

In [157]:
k.sum()

-405334.0

In [ ]:
x1 = stand_x.copy()
for i in range(10,30):
    stand_x = x1
    mask = stand_x == -999
    stand_x[mask] = 0

    pca = PCA(n_components = i)
    stand_x = pca.fit_transform(stand_x)
    stand_x.shape
    
    # instantiate a logistic regression model, and fit with X and y
    model = LogisticRegression()
    model = model.fit(stand_x, y)

    # check the accuracy on the training set
    print(i,",",model.score(stand_x, y))